In [1]:
import splits_generation
import utils
import numpy as np
import pandas as pd

# App1. No sliding window

In [ ]:
# SEPSIS DEFINITION
# Question 1) How many hours do we propagate a True atb backwards and forwards? 
# Parameters defined as Nts_pre and Nts_post. 
# Question 2) How many hours do we propagate from sepsis_onset?
 
sep_def = {
    # Parameteres for antibiotic propagation
    'Nts_pre': 24, 'Nts_post': 24,
    # Parametere for sepsis propagation
    'N_prog_sep': 12, 
    # Parameter for determine sepsis onset
    'increm_sofa': 2,
    # Time step of reference: ICU admission (True) or pre-ICU admission (False)
    'ref_sofa_icu': False
}
 
keys_to_select = ['stay_id', 'stay_time',
'hr_raw','o2sat_raw','temp_raw','sbp_raw','map_raw','dbp_raw','resp_raw','etco2_raw','fio2_raw',
'be_raw', 'bicar_raw','ph_raw','pco2_raw','cl_raw','mg_raw','phos_raw','k_raw','ast_raw','bun_raw',
'alp_raw','ca_raw','crea_raw','bildir_raw','glu_raw','lact_raw', 'bili_raw','tri_raw','hct_raw',
'hgb_raw','ptt_raw','wbc_raw','fgn_raw','plt_raw','age_static','female_static','cai_raw','na_raw',
'po2_raw','alb_raw','alt_raw','ck_raw','ckmb_raw','crp_raw','tnt_raw','urine_raw','basos_raw',
'bnd_raw','eos_raw','esr_raw','hbco_raw','inrpt_raw','lymph_raw','mch_raw','mchc_raw',
'mcv_raw','methb_raw','neut_raw','pt_raw','rbc_raw','rdw_raw','tco2_raw','weight_static','height_static', 
'SI','sep_onset','sep_%2s' % str(sep_def['N_prog_sep'])]

keys_to_select = ['stay_id', 'stay_time','hr_raw', 'o2sat_raw','dbp_raw', 'map_raw', 'resp_raw', 'fio2_raw',  'crp_raw',
                  'po2_raw','bili_raw', 'plt_raw', 'crea_raw',
                  'age_static','female_static','weight_static','height_static', 
                  'sep_onset','sep_%2s' % str(sep_def['N_prog_sep'])]

 
params_to_configure = {
    # File to laod
    'path': '../datasets/eicu_0.5.6.parquet',
    'w_pre_onset':  None,  # Number of windows pre sep_onset = 1
    'w_post_onset':  None,  # Number of windows post sep_onset = 1
    'keys': keys_to_select,
    'label':  ['sep_onset','sep_'+str(sep_def['N_prog_sep'])],
    'f_tr_te':  ['stay_id', 'stay_time','sep_onset','sep_%2s' % str(sep_def['N_prog_sep'])],
    # sliding window
    'moving_span': None,
    # min_length_pat
    'min_length_pat': 0, # 0 value significa calcular
    # Type of imputation
    'imputationType': "LVCF",
    # Threshold: remove patients with less information that theshold value
    'th': 50,
    # Only select patients with sepsis
    "filter_pat": False,
    "length_window": 7,
}
 
seeds = [34, 56, 78]
folders = ["s1", "s2", "s3"]

In [ ]:
keys_by_split = []
idx_exp = 1
for i in range(len(seeds)):
    print("split...", folders[i])
    params_to_configure['min_length_pat'] = 0
    df, min_length_pat = splits_generation.preprocessing(params_to_configure, 
                                                        sep_def,  
                                                        debug=False)
    
    df_data = splits_generation.processing_first_approach(df, params_to_configure)
    
    X_train, X_test, y_train, y_test, keys = splits_generation.get_tr_te_firstapp(df_data, 
                                                                                   params_to_configure,
                                                                                   seeds[i])
        
        
#     df_final = utils.slidingWindow(df, params['moving_span'], min_length_pat)
#     df_filter = utils.filter_windows(df_final, params_to_configure['w_pre_onset'], params_to_configure['w_post_onset'])
#     X_train, X_test, y_train, y_test, keys = splits_generation.get_tr_te(df_filter,
#                                                                         params_to_configure, 
#                                                                         seeds[i])
    
    
    
    keys_by_split.append(keys)
    print("X_train:", X_train.shape)
    print("X_test:", X_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    
    
    np.save(
       "./splits/App"+str(idx_exp)+"/"+ folders[i] + "/X_train_tensor.npy", X_train
    )
    np.save(
       "./splits/App"+str(idx_exp)+"/"+ folders[i] + "/y_train_tensor.npy", y_train
    )
    np.save(
       "./splits/App"+str(idx_exp)+"/"+ folders[i] + "/X_test_tensor.npy", X_test
    )
    np.save(
       "./splits/App"+str(idx_exp)+"/"+ folders[i] + "/y_test_tensor.npy", y_test
    )
    
    pd.DataFrame(keys).to_csv("./splits/App"+str(idx_exp)+"/"+ folders[i] + "/keys.csv")

## App2. Sliding window

In [2]:
# SEPSIS DEFINITION
# Question 1) How many hours do we propagate a True atb backwards and forwards? 
# Parameters defined as Nts_pre and Nts_post. 
# Question 2) How many hours do we propagate from sepsis_onset?
 
sep_def = {
    # Parameteres for antibiotic propagation
    'Nts_pre': 24, 'Nts_post': 24,
    # Parametere for sepsis propagation
    'N_prog_sep': 12, 
    # Parameter for determine sepsis onset
    'increm_sofa': 2,
    # Time step of reference: ICU admission (True) or pre-ICU admission (False)
    'ref_sofa_icu': False
}
 
keys_to_select_filter = ['stay_id', 'stay_time',
'hr_raw','o2sat_raw','temp_raw','sbp_raw','map_raw','dbp_raw','resp_raw','etco2_raw','fio2_raw',
'be_raw', 'bicar_raw','ph_raw','pco2_raw','cl_raw','mg_raw','phos_raw','k_raw','ast_raw','bun_raw',
'alp_raw','ca_raw','crea_raw','bildir_raw','glu_raw','lact_raw', 'bili_raw','tri_raw','hct_raw',
'hgb_raw','ptt_raw','wbc_raw','fgn_raw','plt_raw','age_static','female_static','cai_raw','na_raw',
'po2_raw','alb_raw','alt_raw','ck_raw','ckmb_raw','crp_raw','tnt_raw','urine_raw','basos_raw',
'bnd_raw','eos_raw','esr_raw','hbco_raw','inrpt_raw','lymph_raw','mch_raw','mchc_raw',
'mcv_raw','methb_raw','neut_raw','pt_raw','rbc_raw','rdw_raw','tco2_raw','weight_static','height_static', 'abx', 'sofa']

keys_to_select = ['stay_id', 'stay_time','hr_raw', 'o2sat_raw','dbp_raw', 'map_raw', 'resp_raw', 'fio2_raw',  'crp_raw',
                  'po2_raw','bili_raw', 'plt_raw', 'crea_raw', 'temp_raw',
                  #'age_static','female_static','weight_static','height_static', 
                  'sep_onset','sep_%2s' % str(sep_def['N_prog_sep'])]

 
params_to_configure = {
    # File to laod
    'path': '../datasets/hirid_0.5.6.parquet',
    'w_pre_onset':  None,  # Number of windows pre sep_onset = 1
    'w_post_onset':  None,  # Number of windows post sep_onset = 1
    'keys': keys_to_select,
    'label':  ['sep_onset','sep_'+str(sep_def['N_prog_sep'])],
    'f_tr_te':  ['stay_id', 'stay_time','sep_onset','sep_%2s' % str(sep_def['N_prog_sep']), 'w_id'],
    # sliding window
    'moving_span': 1,
    # min_length_pat
    'min_length_pat': 0, # default: 0
    # Type of imputation
    'imputationType': "LVCF",
    # filter or not patients with less of th information    
    "filter_pat_nans": False, # if true, fix a threshold (next)
    # Threshold: remove patients with less information that theshold value
    'th': 50,
    # Only select patients with sepsis
    "filter_pat": False,
    "length_window": 7,
}
 
seeds = [34, 56, 78]
folders = ["s1", "s2", "s3"]

In [4]:
df, min_length_pat = splits_generation.preprocessing(params_to_configure, 
                                                    sep_def,  
                                                    debug=False)

# of patients: 47827
# of icu-patients: 47827
# of patients pre filter by information: 47827
# of patients post filter by information: 35380
# of patients post imputation: 35380


In [5]:
df_sw = utils.slidingWindow(df, params_to_configure['moving_span'], params_to_configure['length_window'])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/oescudero/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1871144/704914714.py", line 1, in <module>
    df_sw = utils.slidingWindow(df, params_to_configure['moving_span'], params_to_configure['length_window'])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/oescudero/PhD_ICL/AI4MedicalPrediction/Data/utils.py", line 95, in slidingWindow
    df_sw_ttl = pd.concat([df_sw_ttl, df_sw],ignore_index=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/oescudero/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py", line 331, in wrapper
    stacklevel=find_stack_level(),
    ^^^^^^^^^^^^^^^^^^^^^
  File "/home/oescudero/anaconda3/lib/python3.11/site-packages/pandas/core/reshape/concat.py", line 381, in

In [ ]:
df_sw

In [ ]:
# def filterWindows(df_sw):
# Get the unique stay_ids where sep_onset is 1
pats = df_sw[df_sw.sep_onset == 1].stay_id.unique()

# Create a list to store the filtered DataFrames
result_list = []

for pat_id in pats:
    # Filter the DataFrame by each stay_id
    pat = df_sw[df_sw.stay_id == pat_id].reset_index(drop=True)

    # Find the index of the last occurrence of sep_onset = 1
    last_sep_onset_idx = pat[pat['sep_onset'] == 1].index[-1]

    # Get the next w_id after the last sep_onset = 1
    w_id = pat.iloc[last_sep_onset_idx].w_id.split("_")
    next_w_id = w_id[0] + "_" + str(int(w_id[1]) + 1)

    # Check if the next w_id exists in the DataFrame
    if next_w_id in pat['w_id'].values:
        idx = pat[pat['w_id'] == next_w_id].index[0]
        # Filter the DataFrame to include only the rows before the next w_id
        result_df = pat.iloc[:idx]
    else:
        # If the next w_id does not exist, include all rows after the last sep_onset = 1
        result_df = pat.iloc[last_sep_onset_idx + 1:]

    # Add the filtered DataFrame to the results list
    result_list.append(result_df)
    break

# Concatenate all the DataFrames into one
# final_result_df = pd.concat(result_list, ignore_index=True)
#     return final_result_df

In [ ]:
for i in range(len(seeds)):
    print("split...", folders[i])    
    params_to_configure['min_length_pat'] = params_to_configure['length_window']
    X_train, X_test, y_train, y_test, keys, w_id_tr, w_id_te = splits_generation.get_tr_te(df_filter,
                                                                        params_to_configure, 
                                                                        seeds[i])
    
    
    
    keys_by_split.append(keys)
    print("X_train:", X_train.shape)
    print("X_test:", X_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    
    
    np.save(
       "./splits/App"+str(idx_exp)+"/"+ folders[i] + "/X_train_tensor.npy", X_train
    )
    np.save(
       "./splits/App"+str(idx_exp)+"/"+ folders[i] + "/y_train_tensor.npy", y_train
    )
    np.save(
       "./splits/App"+str(idx_exp)+"/"+ folders[i] + "/X_test_tensor.npy", X_test
    )
    np.save(
       "./splits/App"+str(idx_exp)+"/"+ folders[i] + "/y_test_tensor.npy", y_test
    )
    
    pd.DataFrame(keys).to_csv("./splits/App"+str(idx_exp)+"/"+ folders[i] + "/keys.csv")
    w_id_tr.to_csv("./splits/App"+str(idx_exp)+"/"+ folders[i] + "/w_id_tr.csv")
    w_id_te.to_csv("./splits/App"+str(idx_exp)+"/"+ folders[i] + "/w_id_te.csv")

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
pats = df_filter.stay_id.unique()
df_filter_aux = df_filter[df_filter.stay_id.isin(pats[0:100])]
X_train, X_test, y_train, y_test, keys, w_id_tr, w_id_te = get_tr_te(df_filter_aux, params_to_configure, 32, debug=True)

In [ ]:
# def get_tr_te(df, params, seed, debug=True):
df = df_filter_aux.copy()
params = params_to_configure
debug = True
seed = 32
'''
Rules: 
    - Split the same patients into windows. For them, we define a window length as a parameter.
    - The same patient with all windows can only be included in a single split (either train or test).
    - We will allocate 80% for training and 20% for testing.
    - We won't balance the datasets.
'''

# Split the patients into training and testing sets
train_stay_ids, test_stay_ids = \
    train_test_split(df.stay_id.unique(), 
                     test_size=0.2, 
                     random_state=seed)

# Filter the DataFrame based on the stay_ids in the training and testing sets
train_df = df[df['stay_id'].isin(train_stay_ids)].reset_index(drop=True)
y_train_df = train_df[params['f_tr_te']].reset_index(drop=True)
train_df = train_df.drop(params['label'], axis=1)

test_df = df[df['stay_id'].isin(test_stay_ids)].reset_index(drop=True)
y_test_df = test_df[params['f_tr_te']].reset_index(drop=True)
test_df = test_df.drop(params['label'], axis=1)


if debug:
    # Print the number of patients in the training and testing sets
    print("# of windowing patients (train):", len(train_df.w_id.unique()), "- # of original patients:", len(train_df.stay_id.unique()))
    print("# of windowing patients (test):", len(test_df.w_id.unique()), "- # of original patients:", len(test_df.stay_id.unique()))
# Normalization
scaler = StandardScaler()
scaler.fit(train_df[list(train_df.keys())[2:-1]])
X_train_scaled = scaler.transform(train_df[list(train_df.keys())[2:-1]])
X_train_scaled = pd.DataFrame(X_train_scaled, columns=train_df.columns[2:-1])
X_train_scaled = pd.concat([X_train_scaled, train_df[['stay_id', 'stay_time', 'w_id']]], axis=1)

X_test_scaled = scaler.transform(test_df[list(test_df.keys())[2:-1]])
X_test_scaled = pd.DataFrame(X_test_scaled, columns=test_df.columns[2:-1])
X_test_scaled = pd.concat([X_test_scaled, test_df[['stay_id', 'stay_time', 'w_id']]], axis=1)

w_id_tr = X_train_scaled[['stay_id', 'stay_time', 'w_id']]
w_id_te = X_test_scaled[['stay_id', 'stay_time', 'w_id']]

# Convert the DataFrames into tensors while discarding the first two and last features
X_train = utils.dataframe_to_tensor(X_train_scaled, params['min_length_pat'], 'w_id')
X_train = X_train[:, :, 0:X_train.shape[2]-3]
X_test = utils.dataframe_to_tensor(X_test_scaled, params['min_length_pat'], 'w_id')
X_test = X_test[:, :, 0:X_test.shape[2]-3]

y_train = utils.dataframe_to_tensor(y_train_df, params['min_length_pat'], 'w_id')
y_train = y_train[:, :, -3]
y_test = utils.dataframe_to_tensor(y_test_df, params['min_length_pat'], 'w_id')
y_test = y_test[:, :, -3]
# return  X_train, X_test, y_train, y_test, train_df.columns, w_id_tr, w_id_te



In [ ]:
y_test_df

In [ ]:
params_to_configure['min_length_pat'] = params_to_configure['length_window']
X_train, X_test, y_train, y_test, keys, w_id_tr, w_id_te = get_tr_te(df_filter_aux,
                                                                        params_to_configure, 
                                                                        seeds[i])

In [ ]:
df_filter[df_filter.stay_id == 151303]